In [10]:
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SplineConv
from torch_geometric.typing import WITH_TORCH_SPLINE_CONV

from torch_geometric.datasets import Planetoid


In [11]:
if not WITH_TORCH_SPLINE_CONV:
    quit("This example requires 'torch-spline-conv'")

In [17]:
dataset = 'Cora'
transform = T.Compose([
    T.RandomNodeSplit(num_val=500, num_test=500),
    T.TargetIndegree(),
])
# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', dataset)
path = ('./cora')
dataset = Planetoid(path, dataset, transform=transform)
data = dataset[0]

In [18]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SplineConv(dataset.num_features, 16, dim=1, kernel_size=2)
        self.conv2 = SplineConv(16, dataset.num_classes, dim=1, kernel_size=2)

    def forward(self):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.dropout(x, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.log_softmax(x, dim=1)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

In [20]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    log_probs, accs = model(), []
    for _, mask in data('train_mask', 'test_mask'):
        pred = log_probs[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [21]:

for epoch in range(1, 201):
    train()
    train_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.3811, Test: 0.3240
Epoch: 002, Train: 0.4502, Test: 0.3800
Epoch: 003, Train: 0.5521, Test: 0.4840
Epoch: 004, Train: 0.6756, Test: 0.5900
Epoch: 005, Train: 0.7804, Test: 0.6820
Epoch: 006, Train: 0.8296, Test: 0.7400
Epoch: 007, Train: 0.8501, Test: 0.7720
Epoch: 008, Train: 0.8817, Test: 0.8060
Epoch: 009, Train: 0.8958, Test: 0.8300
Epoch: 010, Train: 0.9098, Test: 0.8560
Epoch: 011, Train: 0.9128, Test: 0.8560
Epoch: 012, Train: 0.9192, Test: 0.8580
Epoch: 013, Train: 0.9245, Test: 0.8600
Epoch: 014, Train: 0.9292, Test: 0.8620
Epoch: 015, Train: 0.9309, Test: 0.8640
Epoch: 016, Train: 0.9350, Test: 0.8700
Epoch: 017, Train: 0.9391, Test: 0.8700
Epoch: 018, Train: 0.9420, Test: 0.8680
Epoch: 019, Train: 0.9485, Test: 0.8680
Epoch: 020, Train: 0.9526, Test: 0.8680
Epoch: 021, Train: 0.9549, Test: 0.8640
Epoch: 022, Train: 0.9590, Test: 0.8700
Epoch: 023, Train: 0.9578, Test: 0.8700
Epoch: 024, Train: 0.9602, Test: 0.8660
Epoch: 025, Train: 0.9625, Test: 0.8640
